# **PRIMER MACHINE LEARNING MODEL**

## **PREPARATION**

In [ ]:
# Library
import numpy as np

In [ ]:
# DEFINING VALUES (for future development)
SHORT_LOWER_BOUND = 80
SHORT_UPPER_BOUND = 100
MEDIUM_LOWER_BOUND = 100
MEDIUM_UPPER_BOUND = 150
LONG_LOWER_BOUND = 200
LONG_UPPER_BOUND = 250
PRIMER_LENGTH_LOWER_BOUND = 18
PRIMER_LENGTH_UPPER_BOUND = 22
OPTIMAL_TM_LOWER_BOUND = 58
OPTIMAL_TM_UPPER_BOUND = 60

# DEFINING VALUES (for current development)
EXACTA_SHORT_LENGTH = 95
EXACTA_MEDIUM_LENGTH = 150
EXACTA_LONG_LENGTH = 250
EXACTA_CPG_SHORT = 5
EXACTA_CPG_MEDIUM = 10
EXACTA_CPG_LONG = 15
EXACTA_PRIMER_LENGTH = 22

# DEFINING VALUES (for easier development)
DICT_GENOME_KEY = ["A","C","G","T"]
DICT_GENOME = {"A":0,"C":0,"G":0,"T":0}
KEY_CPG = "CG"

## **TOOLS**

### **TM CALCULATOR**

In [ ]:
def calculateTM(primer):
  counter = DICT_GENOME.copy()
  for i in range(len(primer)):
    counter[primer[i]]+=1
  print(counter)
  return ((4*(counter[DICT_GENOME_KEY[2]]+counter[DICT_GENOME_KEY[1]]))+(2*(counter[DICT_GENOME_KEY[0]]+counter[DICT_GENOME_KEY[3]])))

In [ ]:
# TEST Calculate_Tm
def test_calculateTM():
  res = calculateTM("ACGTACGTACGTACGTAAAACCCGGTT")
  print(res);

test_calculateTM()

{'A': 8, 'C': 7, 'G': 6, 'T': 6}
80


## **PRUNING**

In [ ]:
def findCpg(genome):
  position = []
  for i in range(len(genome)-1):
    if((genome[i]+genome[i+1])==KEY_CPG):
      position.append(i)
  return position